# WeRateDogs Tweeter Archive

## Gather

In [3]:
import pandas as pd
import numpy as np
import requests
import json
import tweepy

In [4]:
tweet = pd.read_csv('twitter-archive-enhanced.csv')

In [5]:
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
if r.status_code == 200:
    with open('image_predictions.tsv', 'wb') as f:
        f.write(r.content)

In [6]:
image = pd.read_csv('image_predictions.tsv', sep = '\t')

In [7]:
###
consumer_key = 'X4Mu5bFyMBpjjstVS4XbYjZcL'
consumer_secret = '6nXJBPj6kBPzT0zP45fZnjiAOuMTN5OEBS3tQ7xKbfIT4NuIst'
access_token = '4366384514-l6MKA0qiPnv018rq4aSQPQd5TODG6Ko4gNeBlSA'
access_secret = '6HcJzhQoY9fZaTfj8p5XXyBdCjNsKiB8SUURkRmSQEh3H'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True )

In [12]:
###
from timeit import default_timer as timer

start = timer()
tweet_id = []
retweet_count = []
favorite_count = []

for ids in image.tweet_id:
    try:
        tweet = api.get_status(ids, tweet_mode='extended')
        tweet_id.append(ids)
        retweet_count.append(tweet.retweet_count)
        favorite_count.append(tweet.favorite_count)
    except Exception as e:
        print(str(e))
        
end = timer()
print(end - start)    

Rate limit reached. Sleeping for: 256
[{u'message': u'No status found with that ID.', u'code': 144}]
Rate limit reached. Sleeping for: 732
[{u'message': u'No status found with that ID.', u'code': 144}]
[{u'message': u'No status found with that ID.', u'code': 144}]
[{u'message': u'No status found with that ID.', u'code': 144}]
[{u'message': u'No status found with that ID.', u'code': 144}]
[{u'message': u'No status found with that ID.', u'code': 144}]
1383.16354203


In [14]:
###
file = open("tweet_json.txt", "w")
for index in range(len(tweet_id)):
    file.write(str(tweet_id[index]) + " " + str(retweet_count[index]) + " " + str(favorite_count[index]) + "\n")
file.close()

In [15]:
info = pd.read_csv('tweet_json.txt', sep=" ", header=None)
info.columns = ['tweet_id','retweet_count','favorite_count']

In [16]:
info.head(10)

,tweet_id,retweet_count,favorite_count
0,666020888022790149,519,2521
1,666029285002620928,47,131
2,666033412701032449,45,125
3,666044226329800704,141,302
4,666049248165822465,40,109
5,666050758794694657,59,133
6,666051853826850816,864,1236
7,666055525042405380,258,441
8,666057090499244032,143,299
9,666058600524156928,58,114


## Assess

In [ ]:
tweet.head(10)

In [ ]:
image.head(10)

In [ ]:
tweet.info()

In [ ]:
image.info()

In [ ]:
all_columns = pd.Series(list(tweet)+ list(image))
all_columns[all_columns.duplicated()]

In [ ]:
list(tweet)

In [ ]:
tweet[tweet['expanded_urls'].isnull()]

In [ ]:
tweet.describe()

In [ ]:
image.describe()

In [ ]:
tweet.sample(5)

In [ ]:
tweet.name.value_counts()

In [ ]:
tweet.source.value_counts()

In [ ]:
tweet[tweet.name.duplicated()]

In [ ]:
tweet.rating_numerator.sort_values()

In [ ]:
tweet.rating_denominator.sort_values()

In [ ]:
len(tweet[(tweet['doggo'] == 'doggo')]) + len(tweet[(tweet['floofer'] == 'floofer')])  + len(tweet[(tweet['pupper'] == 'pupper')]) 
+ len(tweet[(tweet['puppo'] == 'puppo')]) 

#### Quality
##### `patients` table
- Zip code is a float not a string
- Zip code has four digits sometimes
- Tim Neudorf height is 27 in instead of 72 in
- Full state names sometimes, abbreviations other times
- Dsvid Gustafsson
- Missing demographic information (address - contact columns) ***(can't clean)***
- Erroneous datatypes (assigned sex, state, zip_code, and birthdate columns)
- Multiple phone number formats
- Default John Doe data
- Multiple records for Jakobsen, Gersten, Taylor
- kgs instead of lbs for Zaitseva weight

##### `treatments` table
- Missing HbA1c changes
- The letter 'u' in starting and ending doses for Auralin and Novodra
- Lowercase given names and surnames
- Missing records (280 instead of 350)
- Erroneous datatypes (auralin and novodra columns)
- Inaccurate HbA1c changes (leading 4s mistaken as 9s)
- Nulls represented as dashes (-) in auralin and novodra columns

##### `adverse_reactions` table
- Lowercase given names and surnames

#### Tidiness
- Contact column in `patients` table should be split into phone number and email
- Three variables in two columns in `treatments` table (treatment, start dose and end dose)
- Adverse reaction should be part of the `treatments` table
- Given name and surname columns in `patients` table duplicated in `treatments` and `adverse_reactions` tables

## Clean

In [ ]:
patients_clean = patients.copy()
treatments_clean = treatments.copy()
adverse_reactions_clean = adverse_reactions.copy()

### Missing Data

<font color='red'>Complete the following two "Missing Data" **Define, Code, and Test** sequences after watching the *"Address Missing Data First"* video.</font>

#### `treatments`: Missing records (280 instead of 350)

##### Define
*Your definition here. Note: the missing `treatments` records are stored in a file named `treatments_cut.csv`, which you can see in this Jupyter Notebook's dashboard (click the **jupyter** logo in the top lefthand corner of this Notebook). Hint: [documentation page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html) for the function used in the solution.*

Import the cut treatments into a DataFrame and concatenate it with the original treatments DataFrame.

##### Code

In [ ]:
# Your cleaning code here
treatments_cut = pd.read_csv('treatments_cut.csv')
treatments_clean = pd.concat([treatments_clean, treatments_cut],
                             ignore_index=True)

##### Test

In [ ]:
# Your testing code here
treatments_clean.head()

#### `treatments`: Missing HbA1c changes and Inaccurate HbA1c changes (leading 4s mistaken as 9s)
*Note: the "Inaccurate HbA1c changes (leading 4s mistaken as 9s)" observation, which is an accuracy issue and not a completeness issue, is included in this header because it is also fixed by the cleaning operation that fixes the missing "Missing HbA1c changes" observation. Multiple observations in one **Define, Code, and Test** header occurs multiple times in this notebook.*

##### Define
*Your definition here.*

Recalculate the hba1c_change column: hba1c_start minus hba1c_end.

##### Code

In [ ]:
# Your cleaning code here
treatments_clean.hba1c_change = (treatments_clean.hba1c_start - 
                                 treatments_clean.hba1c_end)

##### Test

In [ ]:
# Your testing code here
treatments_clean.hba1c_change.head()

### Tidiness

<font color='red'>Complete the following four "Tidiness" **Define, Code, and Test** sequences after watching the *"Cleaning for Tidiness"* video.</font>

#### Contact column in `patients` table contains two variables: phone number and email

##### Define
*Your definition here. Hint 1: use regular expressions with pandas' [`str.extract` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html). Here is an amazing [regex tutorial](https://regexone.com/). Hint 2: [various phone number regex patterns](https://stackoverflow.com/questions/16699007/regular-expression-to-match-standard-10-digit-phone-number). Hint 3: [email address regex pattern](http://emailregex.com/), which you might need to modify to distinguish the email from the phone number.*

Extract the phone number and email variables from the contact column using regular expressions and pandas' str.extract method. Drop the contact column when done.

##### Code

In [ ]:
# Your cleaning code here
patients_clean['phone_number'] = patients_clean.contact.str.extract('((?:\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4})', expand=True)

# [a-zA-Z] to signify emails in this dataset all start and end with letters
patients_clean['email'] = patients_clean.contact.str.extract('([a-zA-Z][a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[a-zA-Z])', expand=True)

# Note: axis=1 denotes that we are referring to a column, not a row
patients_clean = patients_clean.drop('contact', axis=1)

##### Test

In [ ]:
# Your testing code here

# Confirm contact column is gone
list(patients_clean)

In [ ]:
patients_clean.phone_number.sample(25)

In [ ]:
# Confirm that no emails start with an integer (regex didn't match for this)
patients_clean.email.sort_values().head()

#### Three variables in two columns in `treatments` table (treatment, start dose and end dose)

##### Define
*Your definition here. Hint: use pandas' [melt function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html) and [`str.split()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html). Here is an excellent [`melt` tutorial](https://deparkes.co.uk/2016/10/28/reshape-pandas-data-with-melt/).*

Melt the auralin and novodra columns to a treatment and a dose column (dose will still contain both start and end dose at this point). Then split the dose column on ' - ' to obtain start_dose and end_dose columns. Drop the intermediate dose column.

##### Code

In [ ]:
# Your cleaning code here
treatments_clean = pd.melt(treatments_clean, id_vars=['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change'],
                           var_name='treatment', value_name='dose')
treatments_clean = treatments_clean[treatments_clean.dose != "-"]
treatments_clean['dose_start'], treatments_clean['dose_end'] = treatments_clean['dose'].str.split(' - ', 1).str
treatments_clean = treatments_clean.drop('dose', axis=1)

##### Test

In [ ]:
# Your testing code here
treatments_clean.head()

#### Adverse reaction should be part of the `treatments` table

##### Define
*Your definition here. Hint: [tutorial](https://chrisalbon.com/python/pandas_join_merge_dataframe.html) for the function used in the solution.*

Merge the adverse_reaction column to the treatments table, joining on given name and surname.

##### Code

In [ ]:
# Your cleaning code here
treatments_clean = pd.merge(treatments_clean, adverse_reactions_clean,
                            on=['given_name', 'surname'], how='left')

##### Test

In [ ]:
# Your testing code here
treatments_clean

#### Given name and surname columns in `patients` table duplicated in `treatments` and `adverse_reactions` tables  and Lowercase given names and surnames

##### Define
*Your definition here. Hint: [tutorial](https://chrisalbon.com/python/pandas_join_merge_dataframe.html) for one function used in the solution and [tutorial](http://erikrood.com/Python_References/dropping_rows_cols_pandas.html) for another function used in the solution.*

Adverse reactions table is no longer needed so ignore that part. Isolate the patient ID and names in the patients table, then convert these names to lower case to join with treatments. Then drop the given name and surname columns in the treatments table (so these being lowercase isn't an issue anymore).

##### Code

In [ ]:
# Your cleaning code here
id_names = patients_clean[['patient_id', 'given_name', 'surname']]
id_names.given_name = id_names.given_name.str.lower()
id_names.surname = id_names.surname.str.lower()
treatments_clean = pd.merge(treatments_clean, id_names, on=['given_name', 'surname'])
treatments_clean = treatments_clean.drop(['given_name', 'surname'], axis=1)

##### Test

In [ ]:
# Your testing code here
treatments_clean

In [ ]:
# Patient ID should be the only duplicate column
all_columns = pd.Series(list(patients_clean) + list(treatments_clean))
all_columns[all_columns.duplicated()]

### Quality

<font color='red'>Complete the remaining "Quality" **Define, Code, and Test** sequences after watching the *"Cleaning for Quality"* video.</font>

#### Zip code is a float not a string and Zip code has four digits sometimes

##### Define
*Your definition here. Hint: see the "Data Cleaning Process" page.*

Convert the zip code column's data type from a float to a string using astype, remove the '.0' using string slicing, and pad four digit zip codes with a leading 0.

##### Code

In [ ]:
# Your cleaning code here
patients_clean.zip_code = patients_clean.zip_code.astype(str).str[:-2].str.pad(5, fillchar = '0')
# Reconvert NaNs entries that were converted to '0000n' by code above
patients_clean.zip_code = patients_clean.zip_code.replace('0000n', np.nan)

##### Test

In [ ]:
# Your testing code here
patients_clean.zip_code.head()

#### Tim Neudorf height is 27 in instead of 72 in

##### Define
*Your definition here.*
Replace height for rows in the patients table that have a height of 27 in (there is only one) with 72 in.

##### Code

In [ ]:
# Your cleaning code here
patients_clean.height = patients_clean.height.replace(27, 72)

##### Test

In [ ]:
# Your testing code here
# Should be empty
patients_clean[patients_clean.height == 27]

#### Full state names sometimes, abbreviations other times

##### Define
*Your definition here. Hint: [tutorial](https://chrisalbon.com/python/pandas_apply_operations_to_dataframes.html) for method used in solution.*

Apply a function that converts full state name to state abbreviation for California, New York, Illinois, Florida, and Nebraska.

##### Code

In [ ]:
# Your cleaning code here
# Mapping from full state name to abbreviation
state_abbrev = {'California': 'CA',
                'New York': 'NY',
                'Illinois': 'IL',
                'Florida': 'FL',
                'Nebraska': 'NE'}

# Function to apply
def abbreviate_state(patient):
    if patient['state'] in state_abbrev.keys():
        abbrev = state_abbrev[patient['state']]
        return abbrev
    else:
        return patient['state']
    
patients_clean['state'] = patients_clean.apply(abbreviate_state, axis=1)


##### Test

In [ ]:
# Your testing code here
patients_clean.state.value_counts()

#### Dsvid Gustafsson

##### Define
*Your definition here.*

Replace given name for rows in the patients table that have a given name of 'Dsvid' with 'David'.

##### Code

In [ ]:
# Your cleaning code here
patients_clean.given_name = patients_clean.given_name.replace('Dsvid', 'David')

##### Test

In [ ]:
# Your testing code here
patients_clean[patients_clean.surname == 'Gustafsson']

#### Erroneous datatypes (assigned sex, state, zip_code, and birthdate columns) and Erroneous datatypes (auralin and novodra columns) and The letter 'u' in starting and ending doses for Auralin and Novodra

##### Define
*Your definition here. Hint: [documentation page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html) for one method used in solution, [documentation page](http://pandas.pydata.org/pandas-docs/version/0.20/generated/pandas.to_datetime.html) for one function used in the solution, and [documentation page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.strip.html) for another method used in the solution.*

Convert assigned sex and state to categorical data types. Zip code data type was already addressed above. Convert birthdate to datetime data type. Strip the letter 'u' in start dose and end dose and convert those columns to data type integer.

##### Code

In [ ]:
# Your cleaning code here
# To category
patients_clean.assigned_sex = patients_clean.assigned_sex.astype('category')
patients_clean.state = patients_clean.state.astype('category')

# To datetime
patients_clean.birthdate = pd.to_datetime(patients_clean.birthdate)

# Strip u and to integer
treatments_clean.dose_start = treatments_clean.dose_start.str.strip('u').astype(int)
treatments_clean.dose_end = treatments_clean.dose_end.str.strip('u').astype(int)

##### Test

In [ ]:
# Your testing code here
patients_clean.info()

#### Multiple phone number formats

##### Define
*Your definition here. Hint: helpful [Stack Overflow answer](https://stackoverflow.com/a/123681).*

Strip all " ", "-", "(", ")", and "+" and store each number without any formatting. Pad the phone number with a 1 if the length of the number is 10 digits (we want country code).

##### Code

In [ ]:
# Your cleaning code here
patients_clean.phone_number = patients_clean.phone_number.str.replace(r'\D+', '').str.pad(11, fillchar='1')

##### Test

In [ ]:
# Your testing code here
patients_clean.phone_number.head()

#### Default John Doe data

##### Define
*Your definition here. Recall that it is assumed that the data that this John Doe data displaced is not recoverable.*

Remove the non-recoverable John Doe records from the patients table.

##### Code

In [ ]:
# Your cleaning code here
patients_clean = patients_clean[patients_clean.surname != 'Doe']

##### Test

In [ ]:
# Your testing code here
# Should be no Doe records
patients_clean.surname.value_counts()

In [ ]:
# Should be no 123 Main Street records
patients_clean.address.value_counts()

#### Multiple records for Jakobsen, Gersten, Taylor

##### Define
*Your definition here.*

Remove the Jake Jakobsen, Pat Gersten, and Sandy Taylor rows from the patients table. These are the nicknames, which happen to also not be in the treatments table (removing the wrong name would create a consistency issue between the patients and treatments table). These are all the second occurrence of the duplicate. These are also the only occurences of non-null duplicate addresses.

##### Code

In [ ]:
# Your cleaning code here
# tilde means not: http://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing
patients_clean = patients_clean[~((patients_clean.address.duplicated()) & patients_clean.address.notnull())]

##### Test

In [ ]:
# Your testing code here
patients_clean[patients_clean.surname == 'Jakobsen']

In [ ]:
patients_clean[patients_clean.surname == 'Gersten']

In [ ]:
patients_clean[patients_clean.surname == 'Taylor']

#### kgs instead of lbs for Zaitseva weight

##### Define
*Your definition here.*

Use advanced indexing to isolate the row where the surname is Zaitseva and convert the entry in its weight field from kg to lbs.

##### Code

In [ ]:
# Your cleaning code here
weight_kg = patients_clean.weight.sort_values()[0]
mask = patients_clean.surname == 'Zaitseva'
column_name = 'weight'
patients_clean.loc[mask, column_name] = weight_kg * 2.20462

##### Test

In [ ]:
# Your testing code here
# 48.8 shouldn't be the lowest anymore
patients_clean.weight.sort_values()